In [31]:
import pandas as pd
import sqlalchemy, os
from binance import BinanceSocketManager
from binance.client import Client
from dotenv import load_dotenv
from datetime import datetime, timedelta

load_dotenv()

API_KEY = os.environ.get("API_KEY")
API_SECRET = os.environ.get("API_SECRET")

client = Client(API_KEY, API_SECRET)

In [ ]:
# fetch 1 minute klines for the last day up until now
klines = client.get_historical_klines(
    "BNBBTC", Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC"
)

# fetch 30 minute klines for the last month of 2017
klines = client.get_historical_klines(
    "ETHBTC", Client.KLINE_INTERVAL_30MINUTE, "1 Dec, 2017", "1 Jan, 2018"
)

# fetch weekly klines since it listed
klines = client.get_historical_klines(
    "NEOBTC", Client.KLINE_INTERVAL_1WEEK, "1 Jan, 2017"
)

# [
#     1499040000000,      // 0 Open time
#     "0.01634790",       // 1 Open
#     "0.80000000",       // 2 High
#     "0.01575800",       // 3 Low
#     "0.01577100",       // 4 Close
#     "148976.11427815",  // 5 Volume
#     1499644799999,      // 6 Close time
#     "2434.19055334",    // 7 Quote asset volume
#     308,                // 8 Number of trades
#     "1756.87402397",    // 9 Taker buy base asset volume
#     "28.46694368",      // 10 Taker buy quote asset volume
#     "17928899.62484339" // 11 Ignore.
# ]

In [ ]:
# The following code was in code cell no 2
# Creating Socket Manager
bsm = BinanceSocketManager(client)

# Creating a socket for BTC-USDT pair
# socket = bsm.trade_socket('BTCUSDT')
socket = bsm.trade_socket(PAIR)

# SQLite EngineSQLite Engine
engine = sqlalchemy.create_engine("sqlite:///SLPUSDTstream.db")

### Function for creating dataframe

In [ ]:
def createframe(msg):
    df = pd.DataFrame([msg])
    df = df.loc[:, ["s", "E", "p"]]
    df.columns = ["symbol", "Time", "Price"]
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit="ms")
    return df

In [ ]:
while True:
    await socket.__aenter__()
    msg = await socket.recv()
    while msg["e"] == "error":
        msg = await socket.recv()
    frame = createframe(msg)
    frame.to_sql("SLPUSDT", engine, if_exists="append", index=False)
    print(frame)

In [ ]:
import sqlite3

db = sqlite3.connect('crypto.db')

cursor = db.cursor()

cursor.execute("""
    CREATE TABLE inventory (
        symbol TEXT,
        avg_price REAL
    )""")
db.commit()
db.close()

In [ ]:
db = sqlite3.connect('crypto.db')
cursor = db.cursor()
cursor.execute("INSERT INTO inventory VALUES ('BTCUSDT',30000.24)")
db.commit()
db.close()

In [ ]:
def fetchall(selection='*'):
    db = sqlite3.connect('crypto.db')
    cursor = db.cursor()
    cursor.execute(f"SELECT {selection} FROM inventory")
    alldata = cursor.fetchall()
    # print(alldata)
    db.commit()
    db.close()
    return alldata

In [ ]:
data = [
    ('OMUSDT', 352.53),
    ('ENSUSDT',12.43)
]
db = sqlite3.connect('crypto.db')
cursor = db.cursor()
print(cursor.executemany("INSERT INTO inventory VALUES (?,?)",data))
db.commit()
db.close()

In [ ]:
example = [item[0] for item in fetchall('avg_price')]
print(example)

In [ ]:
i = 1
dictionary = {}
db = sqlite3.connect('crypto.db')
cursor = db.cursor()
cursor.execute(f"SELECT * FROM inventory")
alldata = cursor.fetchall()
symbols = [x[i] for x in alldata]
print(symbols)
for data in alldata:
    print(data[0])
db.commit()
db.close()

In [ ]:
def fetchall():
    db = sqlite3.connect('crypto.db')
    cursor = db.cursor()
    cursor.execute("SELECT * FROM inventory")
    alldata = cursor.fetchall()
    db.commit()
    db.close()
    return alldata

def extract_for_tuple(alldata, i = 0):
    symbols = [x[i] for x in alldata]
    return symbols

def get_buy_price(symbol):
    db = sqlite3.connect('crypto.db')
    cursor = db.cursor()
    cursor.execute(f"SELECT avg_price FROM inventory WHERE symbol = '{symbol}'")
    price = cursor.fetchone()[0]
    db.commit()
    db.close()
    return price

print(extract_for_tuple(fetchall(),1))
dictionary = {}
# for price in 
print(get_buy_price('BTCUSDT'))

In [ ]:
def filter_symbol(level, list_of_symbols, threshold, window):
    from datetime import datetime, timedelta

    selected_symbols = []

    for symbol in list_of_symbols:

        klines = client.get_historical_klines(
            symbol,
            Client.KLINE_INTERVAL_1MINUTE,
            str(datetime.now() - timedelta(hours=window)),
            str(datetime.now()),
        )

        percent_change = get_percent_change(klines[0], klines[-1])

        if percent_change > threshold:
            selected_symbols.append(symbol)
            print(symbol, percent_change)
            
    with open("pairs_level_"+str(level)+".txt", "w") as file:
        file.write(str(selected_symbols)[1:-1].replace('\'','').replace(' ',''))
        
    return selected_symbols

import time
import pandas as pd
import sqlalchemy, os
from binance import BinanceSocketManager
from binance.client import Client
from dotenv import load_dotenv
from datetime import datetime, timedelta

load_dotenv()

API_KEY = os.environ.get("API_KEY")
API_SECRET = os.environ.get("API_SECRET")

# Binance Client
client = Client(API_KEY, API_SECRET)

In [ ]:
klines = client.get_historical_klines(
            'BTCBUSD',
            Client.KLINE_INTERVAL_1MINUTE,
            str(datetime.now() - timedelta(minutes=361)),
            str(datetime.now()),
        )
print(len(klines))
for kline in klines:
    print(kline[0],kline[6])

In [ ]:
list_symbols_USDT = [item["symbol"] for item in client.get_exchange_info()["symbols"] if ("USDT" in item["symbol"]) & ("DOWN" not in item["symbol"])]

In [22]:
# Function for printing to files
def print_file(ls, file):
    with open(str(file)+'.txt','w') as file:
        file.write(str(ls)[1:-1].replace(' ','').replace('\'',''))
        
# Function for getting price
def get_price(ls):
    avg = (float(ls[1]) + float(ls[4])) / 2
    price = float("{:.4f}".format(avg))
    return price

# Function for getting percent change in 6 hour
def get_percent_change(ls_1, ls_2):
    price_1 = get_price(ls_1)
    price_2 = get_price(ls_2)

    change = (price_2 - price_1) / price_1

    change_in_percent = float("{:.4f}".format(change * 100))

    return change_in_percent

In [15]:
ls = [item for item in [item['symbol'] for item in client.get_ticker() if float(item['priceChangePercent'])>10.0] if ('USDT' in item) & ('DOWN' not in item)]
print_file(ls,'pairs_level_1')

In [18]:
level1 = [item for item in [item['symbol'] for item in client.get_ticker() if float(item['priceChangePercent'])>10.0] if ('USDT' in item) & ('DOWN' not in item)]
print(level1)
# for item in info:
#     print(type(item['priceChangePercent']))

['ADAUSDT', 'LINKUSDT', 'FETUSDT', 'THETAUSDT', 'DUSKUSDT', 'MTLUSDT', 'TOMOUSDT', 'PERLUSDT', 'NKNUSDT', 'ARPAUSDT', 'RLCUSDT', 'TCTUSDT', 'BTSUSDT', 'COTIUSDT', 'DATAUSDT', 'PNTUSDT', 'ADAUPUSDT', 'LINKUPUSDT', 'DGBUSDT', 'DCRUSDT', 'BLZUSDT', 'OCEANUSDT', 'RSRUSDT', 'TRBUSDT', 'DIAUSDT', 'BELUSDT', 'WINGUSDT', 'FLMUSDT', 'XVSUSDT', 'ALPHAUSDT', 'AUDIOUSDT', 'HARDUSDT', 'UNFIUSDT', 'TRUUSDT', 'LITUSDT', 'DODOUSDT', 'FISUSDT', 'PONDUSDT', 'DEGOUSDT', 'LINAUSDT', 'BAKEUSDT', 'ARUSDT', 'ATAUSDT', 'GTCUSDT', 'DEXEUSDT', 'VIDTUSDT', 'FRONTUSDT', 'CVPUSDT', 'CHESSUSDT', 'WOOUSDT']


In [53]:
def filter_symbol(level, list_of_symbols, threshold, time):

    selected_symbols = []

    for symbol in list_of_symbols:
        klines = client.get_historical_klines(
            symbol,
            Client.KLINE_INTERVAL_1MINUTE,
            str(datetime.now() - time),
            str(datetime.now()),
        )
        
        percent_change = get_percent_change(klines[0], klines[-1])

        if percent_change > threshold:
            selected_symbols.append(symbol)
            print(symbol,percent_change)
            
    # with open("pairs_level_"+str(level)+".txt", "w") as file:
    print_file(selected_symbols,'pairs_level_'+str(level))
        
    return selected_symbols

In [ ]:
def filter_symbol(level, list_of_symbols, threshold, time):
    selected_symbols = []

    for symbol in list_of_symbols:
        klines = client.get_historical_klines(
            symbol,
            Client.KLINE_INTERVAL_1MINUTE,
            str(datetime.now() - time),
            str(datetime.now()),
        )

        percent_change = get_percent_change(klines[0], klines[-1])

        if percent_change > threshold:
            selected_symbols.append(symbol)
            print(symbol,percent_change)

    print(selected_symbols)

In [26]:
def filter_symbol(level, list_of_symbols, threshold, window):
    print('pairs_level_'+str(level))

In [54]:
level2 = filter_symbol(2, level1, 6, timedelta(hours=12))

BLZUSDT 10.4269
WINGUSDT 44.5155
FLMUSDT 27.1825
LITUSDT 8.3515
DEGOUSDT 15.5015
ATAUSDT 9.3857


In [56]:
klines = client.get_historical_klines(
    'THETAUSDT',
    Client.KLINE_INTERVAL_1MINUTE,
    str(datetime.now() - timedelta(minutes=390)),
    str(datetime.now()),
)
print(len(klines))
print(klines[0])
print(klines[-1])

30
[1654677480000, '1.34700000', '1.34800000', '1.34600000', '1.34800000', '1003.30000000', 1654677539999, '1351.48640000', 8, '801.80000000', '1080.08930000', '0']
[1654679220000, '1.32500000', '1.32800000', '1.32400000', '1.32600000', '4207.80000000', 1654679279999, '5577.43230000', 45, '2303.90000000', '3053.68590000', '0']
